In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("uraninjo/augmented-alzheimer-mri-dataset")

# print("Path to dataset files:", path)

In [2]:
# !pip install -r requirements.txt

In [3]:
heavy_models = [
    'vit_base_patch16_224.augreg2_in21k_ft_in1k',
    'vit_base_patch16_224',
    "vit_tiny_patch16_224.augreg_in21k_ft_in1k",
    "vit_tiny_patch16_224",
    'swin_base_patch4_window7_224.ms_in22k_ft_in1k',
    'swin_base_patch4_window7_224',
    'maxvit_tiny_224',
    
    'tf_efficientnet_b4.ns_jft_in1k',  # 380x380 resolution
    'convnext_small.fb_in22k_ft_in1k'  # Deep modern CNN
]

medium_models = [
    'tf_efficientnetv2_s.in21k_ft_in1k', # 300x300 resolution
    'convnext_tiny.fb_in22k_ft_in1k',
    'coatnet_0_rw_224.sw_in1k',
    
    'resnet50.a1_in1k',
    'resnext50_32x4d.a1h_in1k',
    'densenet121.ra_in1k',
    'inception_v3',
    'xception',
    'vgg16_bn'
]

light_models = [
    'mobilevit_s.cvnets_in1k',
    'efficientformer_l1.snap_dist_in1k',
    'poolformer_s12.sail_in1k',

    'resnet18',
    'efficientnet_b0',
    'mobilenetv3_large_100.ra_in1k',
    'ghostnet_100.in1k'
]

def get_model_batches():
    """
    Returns a list of dictionaries. Each dictionary contains:
    - 'models': List of model names
    - 'config': Dictionary of training configurations (batch_size, epochs, img_size)
    """
    
    batches = []
    
    # Helper to chunk lists if they are too long for one subprocess
    def chunk_list(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    # Create Batches
    for chunk in chunk_list(heavy_models, 3): # Run 3 heavy models per subprocess
        batches.append(chunk)
    
    for chunk in chunk_list(medium_models, 5): # Run 5 medium models per subprocess
        batches.append(chunk)
    
    for chunk in chunk_list(light_models, 8): # Run 5 medium models per subprocess
        batches.append(chunk)

    return batches

In [4]:
import subprocess
import sys
import os
from hf_manager import HFBackupManager
import traceback
from datetime import datetime
import warnings

warnings.filterwarnings("ignore")

RESULTS_DIR = "output" # The folder your subprocess writes to

In [5]:
SUBPROCESS_TEMPLATE = r"""
import sys
from .cross_validation import Cross_Validator
from .utils import Logger

def run_batch():
    print("--- Subprocess Started ---")
    
    # Placeholders replaced by the main script
    models = __models_list__
    use_aug = __augmentation__

    # logic
    logger = Logger("05_12_2025")
    logger.info(f"Starting Validation for {models}")
    
    validator = Cross_Validator(models, logger, use_aug = use_aug)
    validator.run()
    
    logger.info("Validation Complete")

if __name__ == "__main__":
    run_batch()
"""

In [ ]:
def run_subprocess(models_list, use_aug):
    # Name of the temporary file inside the package
    # It must be inside 'module' folder to work with -m module.name
    script_filename = "temp_runner.py"
    script_path = os.path.join("module", script_filename)
    
    # 2. FILL TEMPLATE & WRITE TO DISK
    script_content = SUBPROCESS_TEMPLATE.replace("__models_list__", str(models_list)) \
                                        .replace("__augmentation__", str(use_aug))
    
    with open(script_path, "w") as f:
        f.write(script_content)
    
    print(f"🚀 Launching subprocess for: {models_list}")

    try:
        # 3. RUN AS MODULE
        # Syntax: python -m package_name.script_name_without_extension
        # We run this from the root directory (parent of 'module')
        module_path = f"module.{script_filename[:-3]}" # removes .py
        
        subprocess.run(
            [sys.executable, "-m", module_path], 
            check=True
        )
    finally:
        # 4. CLEANUP
        if os.path.exists(script_path):
            os.remove(script_path)
            print("🧹 Cleanup complete.")

In [6]:
def run_queue(use_aug):
    # 1. GENERATE RUN ID
    run_id = f"AUG_{use_aug}_RUN_{datetime.now().strftime('%Y%m%d_%H%M')}"
    
    # 2. STATE TRACKING
    total_batches = 0
    completed_batches = 0
    run_status = "Success"
    errors = []
    
    print(f"--- Starting Queue {run_id} ---")

    try:
        batches = get_model_batches()
        total_batches = len(batches)

        for i, batch in enumerate(batches):
            batch_id = i + 1
            print(f"\n>>> [Master] Launching Batch {batch_id} / {total_batches}")
            print(f"Models: {batch}")

            # --- A. RUN SUBPROCESS ---
            try:
                run_subprocess(batch, use_aug)
                
            except subprocess.CalledProcessError as e:
                print(f"❌ Batch {batch_id} crashed: {e}")
                errors.append(f"Batch {batch_id} Process Error: {str(e)}")
                continue 

    except KeyboardInterrupt:
        print("\n\n⚠️ Interrupted by User!")
        run_status = "Stopped by User"
        
    except Exception as e:
        print(f"\n\n❌ Critical Master Script Error: {e}")
        run_status = "Crashed"
        errors.append(traceback.format_exc())

    finally:
        # --- UPLOAD STATUS TO HUGGING FACE ---
        print("\n" + "="*50)
        print("🛑 EXECUTION ENDED - UPLOADING LOG")
        print("="*50)

        if not errors :
            error_text = "None"
        else :
            error_text = "\n".join(f"\n--- Error {i}\n{e}" for i, e in enumerate(errors, start = 1))
        
        remaining = total_batches - completed_batches
        summary_text = f"""
=== TRAINING RUN SUMMARY ===
Run ID: {run_id}
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Final Status: {run_status}

Batches Uploaded: {completed_batches}
Batches Failed/Skipped: {remaining}
Total Scheduled: {total_batches}

------ Error Logs ------
{error_text}
        """
        
        filename = f"REPORT_{run_id}.txt"
        
        with open(filename, "w", encoding = "utf-8") as f:
            f.write(summary_text)

In [ ]:
run_queue(False)
run_queue(True)

--- Starting Queue AUG_False_RUN_20251205_1819 ---

>>> [Master] Launching Batch 1 / 6
Models: ['vit_base_patch16_224.augreg2_in21k_ft_in1k', 'vit_base_patch16_224', 'vit_tiny_patch16_224.augreg_in21k_ft_in1k']
🚀 Launching subprocess for: ['vit_base_patch16_224.augreg2_in21k_ft_in1k', 'vit_base_patch16_224', 'vit_tiny_patch16_224.augreg_in21k_ft_in1k']
